In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import statsmodels.formula.api as smf

# Question 2

In [2]:
train = pd.read_csv("yelp142_train.csv")
# len(train)
train.head()

,stars,review_count,GoodForKids,Alcohol,BusinessAcceptsCreditCards,WiFi,BikeParking,ByAppointmentOnly,WheelechairAccessible,OutdoorSeating,RestaurantsReservations,DogsAllowed,Caters
0,3.5,26,(Missing),(Missing),TRUE,(Missing),TRUE,(Missing),(Missing),(Missing),(Missing),(Missing),(Missing)
1,3.0,40,TRUE,'full_bar',TRUE,'free',FALSE,(Missing),(Missing),TRUE,FALSE,(Missing),(Missing)
2,4.5,45,TRUE,'full_bar',TRUE,'free',FALSE,(Missing),(Missing),TRUE,FALSE,(Missing),(Missing)
3,4.5,61,TRUE,(Missing),TRUE,(Missing),(Missing),(Missing),(Missing),FALSE,(Missing),(Missing),FALSE
4,3.5,5,TRUE,'beer_and_wine',TRUE,(Missing),(Missing),(Missing),(Missing),FALSE,FALSE,(Missing),(Missing)


In [3]:
test = pd.read_csv("yelp142_test.csv")
# len(train)
test.head()

,stars,review_count,GoodForKids,Alcohol,BusinessAcceptsCreditCards,WiFi,BikeParking,ByAppointmentOnly,WheelechairAccessible,OutdoorSeating,RestaurantsReservations,DogsAllowed,Caters
0,4.5,11,(Missing),(Missing),TRUE,(Missing),(Missing),(Missing),(Missing),(Missing),FALSE,(Missing),(Missing)
1,4.0,82,TRUE,'none',TRUE,'no',TRUE,(Missing),TRUE,FALSE,FALSE,(Missing),TRUE
2,5.0,3,(Missing),(Missing),TRUE,'no',TRUE,TRUE,TRUE,(Missing),(Missing),FALSE,(Missing)
3,4.0,141,TRUE,'none',TRUE,'no',FALSE,(Missing),TRUE,FALSE,FALSE,(Missing),TRUE
4,5.0,222,TRUE,'none',TRUE,'free',TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE


# b. i)

In [4]:
# ols = smf.ols(formula = "stars ~ review_count + C(GoodForKids, Treatment(reference=’(Missing)’))", data = datatrain)
# linreg = """stars ~ review_count + 
#         C(GoodForKids, Treatment(reference="(Missing)")) +
#         C(Alcohol, Treatment(reference="(Missing)")) +
#         C(BusinessAcceptsCreditCards, Treatment(reference="(Missing)")) +
#         C(WiFi, Treatment(reference="(Missing)")) +
#         C(BikeParking, Treatment(reference="(Missing)")) +
#         C(ByAppointmentOnly, Treatment(reference="(Missing)")) +
#         C(WheelechairAccessible, Treatment(reference="(Missing)")) +
#         C(OutdoorSeating, Treatment(reference="(Missing)")) +
#         C(RestaurantsReservations, Treatment(reference="(Missing)")) +
#         C(DogsAllowed, Treatment(reference="(Missing)")) +
#         C(Caters, Treatment(reference="(Missing)"))"""
# print(linreg)

ols = smf.ols(formula='stars ~ review_count + \
                C(GoodForKids,Treatment(reference="(Missing)")) + \
                C(Alcohol, Treatment(reference="(Missing)")) + \
                C(BusinessAcceptsCreditCards,Treatment(reference="(Missing)")) + \
                C(WiFi, Treatment(reference="(Missing)")) + \
                C(BikeParking,Treatment(reference="(Missing)")) + \
                C(ByAppointmentOnly,Treatment(reference="(Missing)")) + \
                C(WheelechairAccessible, Treatment(reference="(Missing)")) + \
                C(OutdoorSeating,Treatment(reference="(Missing)")) + \
                C(RestaurantsReservations, Treatment(reference="(Missing)")) + \
                C(DogsAllowed,Treatment(reference="(Missing)")) + \
                C(Caters, Treatment(reference="(Missing)"))', data=train)
model = ols.fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  stars   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     47.80
Date:                Sat, 13 Mar 2021   Prob (F-statistic):          3.10e-215
Time:                        17:14:41   Log-Likelihood:                -7317.5
No. Observations:                6272   AIC:                         1.469e+04
Df Residuals:                    6246   BIC:                         1.486e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

# b. ii)

In [5]:
x_train = pd.get_dummies(train.loc[:, 'review_count':])
x_test = pd.get_dummies(test.loc[:, 'review_count':])

y_train = train['stars']
y_test = test['stars']

In [6]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

dectree = DecisionTreeRegressor()
grid_values = {'ccp_alpha': np.linspace(0, 0.003, 51),
               'min_samples_leaf': [5],
               'min_samples_split': [20],
               'random_state': [88]}
dectree_cv = GridSearchCV(dectree, param_grid = grid_values, cv = KFold(n_splits = 5,random_state = 1,shuffle = True)) 
dectree_cv.fit(x_train, y_train)
ccp = dectree_cv.best_params_['ccp_alpha']
ccp

0.00156

# b. iii)

In [7]:
from sklearn.metrics import mean_absolute_error 

def osr2_ols(model, df_train, df_test, dependent_var): 
    y_pred = model.predict(df_test)
    y_test = df_test[dependent_var]
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(df_train[dependent_var]))**2) 
    return (1 - SSE/SST)

print('Lin Model OSR2:', round(osr2_ols(model, train, test, 'stars'), 5))
print('Lin Model MAE:', round(mean_absolute_error(y_test, model.predict(test)), 5))

Lin Model OSR2: 0.18042
Lin Model MAE: 0.61709


In [8]:
def osr2_cart(model, x_test, y_test, y_train): 
    y_pred = model.predict(x_test)
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(y_train))**2)
    return (1 - SSE/SST)

print('Cart Model OSR2:', round(osr2_cart(dectree_cv, x_test, y_test, y_train), 5))
print('Cart Model MAE:', round(mean_absolute_error(y_test, dectree_cv.predict(x_test)),5))

Cart Model OSR2: 0.1732
Cart Model MAE: 0.61476


# c

In [9]:
train['fourOrAbove'] = np.where(train['stars'] >= 4, 1, 0)
test['fourOrAbove'] = np.where(test['stars'] >= 4, 1, 0)

# d. i)

I think this is reasonable because false positives and false negatives can sometimes skew the results and so it is better to just focus on the accuracy level becaue it will be the most helpful for the model

# d. ii)

In [10]:
y_pred_ols = model.predict(test)
y_new_ols_pred = pd.Series([1 if x >= 4 else 0 for x in y_pred_ols])

y_pred_dectree = dectree_cv.predict(x_test)
y_new_dtr_pred = pd.Series([1 if x >= 4 else 0 for x in y_pred_dectree])

# d.iii)

In [ ]:
logistic = smf.logit(formula = 'fourOrAbove ~ stars + review_count + \
                C(GoodForKids,Treatment(reference="(Missing)")) + \
                C(Alcohol, Treatment(reference="(Missing)")) + \
                C(BusinessAcceptsCreditCards,Treatment(reference="(Missing)")) + \
                C(WiFi, Treatment(reference="(Missing)")) + \
                C(BikeParking,Treatment(reference="(Missing)")) + \
                C(ByAppointmentOnly,Treatment(reference="(Missing)")) + \
                C(WheelechairAccessible, Treatment(reference="(Missing)")) + \
                C(OutdoorSeating,Treatment(reference="(Missing)")) + \
                C(RestaurantsReservations, Treatment(reference="(Missing)")) + \
                C(DogsAllowed,Treatment(reference="(Missing)")) + \
                C(Caters, Treatment(reference="(Missing)"))', data = train)
logreg = logistic.fit()
print(logreg.summary())

The kernel kept dying when I ran this, not exactly sure why but this is how you make a logistic regression model, similar to linear regression but with smf.logit()

# d.iv)

In [11]:
x_train2 = pd.get_dummies(train.drop(['fourOrAbove'], axis = 1))
x_test2 = pd.get_dummies(test.drop(['fourOrAbove'], axis = 1))

In [12]:
y_train2 = train['fourOrAbove']
y_test2 = test['fourOrAbove']

In [13]:
grid_values = {'ccp_alpha': np.linspace(0, 0.09, 201),'min_samples_leaf': [5],'min_samples_split': [20],
               'max_depth': [30],'class_weight' : [{0: 1, 1: 20}],'random_state': [88]}
dectree = DecisionTreeClassifier()
dectree_cv = GridSearchCV(dectree, param_grid = grid_values, verbose = 1)
dectree_cv.fit(x_train2, y_train2)
ccp2 = dectree_cv.best_params_['ccp_alpha']
ccp2

Fitting 5 folds for each of 201 candidates, totalling 1005 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1005 out of 1005 | elapsed:   14.4s finished


0.0

# d.v)

In [14]:
from sklearn.ensemble import RandomForestClassifier
randforest = RandomForestClassifier(max_features=len(x_train.columns),min_samples_leaf=5,n_estimators = 500, random_state=88, verbose=2)
randforest.fit(x_train2, y_train2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
b

building tree 377 of 500
building tree 378 of 500
building tree 379 of 500
building tree 380 of 500
building tree 381 of 500
building tree 382 of 500
building tree 383 of 500
building tree 384 of 500
building tree 385 of 500
building tree 386 of 500
building tree 387 of 500
building tree 388 of 500
building tree 389 of 500
building tree 390 of 500
building tree 391 of 500
building tree 392 of 500
building tree 393 of 500
building tree 394 of 500
building tree 395 of 500
building tree 396 of 500
building tree 397 of 500
building tree 398 of 500
building tree 399 of 500
building tree 400 of 500
building tree 401 of 500
building tree 402 of 500
building tree 403 of 500
building tree 404 of 500
building tree 405 of 500
building tree 406 of 500
building tree 407 of 500
building tree 408 of 500
building tree 409 of 500
building tree 410 of 500
building tree 411 of 500
building tree 412 of 500
building tree 413 of 500
building tree 414 of 500
building tree 415 of 500
building tree 416 of 500


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    1.9s finished


RandomForestClassifier(max_features=36, min_samples_leaf=5, n_estimators=500,
                       random_state=88, verbose=2)

In [ ]:
grid_values = {'max_features': np.linspace(1,17,17, dtype='int32'),
               'min_samples_leaf': [5],
               'n_estimators': [500],
               'random_state': [88]}
randfor2 = RandomForestClassifier()
cv = KFold(n_splits = 5, random_state = 300, shuffle = True)
randforest_cv = GridSearchCV(randfor2, param_grid = grid_values, cv = cv,verbose = 2) 
randforest_cv.fit(x_train2, y_train2)

Fitting 5 folds for each of 17 candidates, totalling 85 fits
[CV] max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88, total=   2.1s
[CV] max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV]  max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.5s
[CV] max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88 
[CV]  max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.6s
[CV] max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88 
[CV]  max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.4s
[CV] max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88 
[CV]  max_features=1, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.5s
[CV] max_features=2, min_samples_leaf=5, n_estimators=500, random_state=88 
[CV]  max_features=2, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.6s
[CV] max_features=2, min_samples_leaf=5, n_estimators=500, random_state=88 
[CV]  max_features=2, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.6s
[CV] max_features=2, min_samples_leaf=5, n_estimators=500, random_state=88

[CV]  max_features=11, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.6s
[CV] max_features=11, min_samples_leaf=5, n_estimators=500, random_state=88 
[CV]  max_features=11, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.5s
[CV] max_features=11, min_samples_leaf=5, n_estimators=500, random_state=88 
[CV]  max_features=11, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.5s
[CV] max_features=11, min_samples_leaf=5, n_estimators=500, random_state=88 
[CV]  max_features=11, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.5s
[CV] max_features=11, min_samples_leaf=5, n_estimators=500, random_state=88 
[CV]  max_features=11, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.6s
[CV] max_features=12, min_samples_leaf=5, n_estimators=500, random_state=88 
[CV]  max_features=12, min_samples_leaf=5, n_estimators=500, random_state=88, total=   1.5s
[CV] max_features=12, min_samples_leaf=5, n_estimators=500, ran

[Parallel(n_jobs=1)]: Done  85 out of  85 | elapsed:  2.6min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=300, shuffle=True),
             estimator=RandomForestClassifier(),
             param_grid={'max_features': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
      dtype=int32),
                         'min_samples_leaf': [5], 'n_estimators': [500],
                         'random_state': [88]},
             verbose=2)

# d. vii - couldnt get this

# e

As a data scientist advising restaurant managers to better increase their chance of having high rated reviews on yelp, I would advise these 3 things:    
1.When you fill out your restaurant's business profile on Yelp completely, including details about parking and outdoor seating, etc.                 
2.You should respond to both negative and positive Yelp reviews. It will help you connect with the customers better, and how that you are willing to improve your service and cater to customer needs.                                   
3.Give exclusive deals or incentives to customers so they are motivated to leave reviews on Yelp